In [ ]:
#Exercise 0

In [12]:
def github() -> str:

    return "https://github.com/rohund/ProblemSet/blob/main/ProblemSet6.py"

In [13]:
#Exercise 1

In [19]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from sqlalchemy import text

path = "/home/jovyan/econ-481-jupyterhub/auctions.db"

class DataBase:
    def __init__(self, loc: str, db_type: str = "sqlite") -> None:
        """Initialize the class and connect to the database"""
        self.loc = loc
        self.db_type = db_type
        self.engine = create_engine(f'{self.db_type}:///{self.loc}')
    def query(self, q: str) -> pd.DataFrame:
        """Run a query against the database and return a DataFrame"""
        with Session(self.engine) as session:
            df = pd.read_sql(q, session.bind)
        return(df)
    def execute(self, q: str) -> None:
        """Execute statement on the database"""
        with self.engine.connect() as conn:
            conn.execute(text(q))

auctions = DataBase(path)
query = """
select itemid,
sqrt(sum((bidamount - avg_bid) * (bidamount - avg_bid)) / (count(*) - 1)) as std
from (select itemid,
    bidamount,
    avg(bidamount) over (partition by itemid) as avg_bid
    from bids)
group by itemid
having count(*) > 1
"""
print(auctions.query(query))

       itemid       std
0   172998011  0.000000
1   174445924  0.577350
2   174551689  1.830034
3   174767945  8.607399
4   174871788  3.109126
..        ...       ...
82  182571641  7.767700
83  182660924  4.364634
84  182745253  2.884098
85  182883873  0.707107
86  182925212  1.154701

[87 rows x 2 columns]


In [20]:
#Exercsise 2

In [21]:
def bidder_spend_frac() -> str:
    q = """
    select b.biddername
    , b.total_bids as total_bids
    , case when s.total_spend is null then 0 else s.total_spend end as total_spend
    , case when s.total_spend is null then 0 else s.total_spend end / b.total_bids as spend_frac
    from (
        select biddername
        , sum(maxbid) as total_bids
        from (
            select biddername
            , itemid
            , max(bidamount) as maxbid
            from bids
            group by biddername, itemid
            )
        group by biddername) as b
    left join
        (
        select biddername
        , sum(maxbid) as total_spend
        from (
            select biddername
            , max(bidamount) as maxbid
            from bids
            group by itemid
        )
        group by biddername) as s
    on b.biddername = s.biddername
    """
    return q
   
   

In [22]:
#Exercise 3

In [23]:
def min_increment_freq() -> str:
    
    query = """
    WITH ranked_bids AS (
        SELECT
            b.itemid,
            b.bidderName,
            b.bidamount,
            i.bidIncrement,
            ROW_NUMBER() OVER (PARTITION BY b.itemid ORDER BY b.bidamount) AS rn
        FROM
            bids b
        JOIN
            items i ON b.itemid = i.itemid
        WHERE
            i.isBuyNowUsed = 0
    ),
    prev_bids AS (
        SELECT
            rb.itemid,
            rb.bidderName,
            rb.bidamount,
            rb.bidIncrement,
            rb.rn,
            LAG(rb.bidamount) OVER (PARTITION BY rb.itemid ORDER BY rb.rn) AS prev_bid
        FROM
            ranked_bids rb
    ),
    increment_bids AS (
        SELECT
            itemid,
            bidderName,
            bidamount,
            bidIncrement,
            prev_bid
        FROM
            prev_bids
        WHERE
            prev_bid IS NOT NULL
            AND bidamount = prev_bid + bidIncrement
    )
    SELECT
        COUNT(*) * 1.0 / (SELECT COUNT(*) FROM bids b JOIN items i ON b.itemid = i.itemid WHERE i.isBuyNowUsed = 0) AS freq
    FROM
        increment_bids;
    """
    return query
auctions = DataBase(path)
min_increment_freq_query = min_increment_freq()
result_df = auctions.query(min_increment_freq_query)
print(result_df)


       freq
0  0.184502


In [24]:
#Exercise 4

In [25]:
def win_perc_by_timestamp() -> str:
    q = """
    select time_norm as timestamp_bin
    , avg(iswinbid) as win_perc
    from (
        with a as (
            select itemid, starttime, endtime,
            julianday(endtime) - julianday(starttime) as length
            from items
        )
        select b.itemid, b.bidtime, a.starttime, a.endtime,
        cast(((julianday(endtime)-julianday(bidtime)) / a.length)*10+1 as integar) as time_norm
        , b.bidamount == max(bidamount) over (partition by b.itemid) as iswinbid
        from bids as b
        inner join a
        on b.itemid=a.itemid
        )
    group by timestamp_bin
    """
    return q